<a href="https://colab.research.google.com/github/Sanazzzmi/Sentiment-analysis-of-the-Persian-text-with-Rule-CNN/blob/main/Sentiment_analysis_of_the_Persian_text_with_Rule_%26_CNN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### install library & import data

In [1]:
#Article idea => https://www.sciencedirect.com/science/article/abs/pii/S0925231219313815

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install fasttext

!pip install hazm

!pip install deplacy hazm
!test -f resources-0.5.zip || curl -LO https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip
!test -d resources || ( mkdir -p resources && cd resources && unzip ../resources-0.5.zip )
import hazm

from __future__ import unicode_literals
from hazm import *
import random

     |████████████████████████████████| 68 kB 3.0 MB/s 
  Using cached pybind11-2.9.0-py2.py3-none-any.whl (210 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3127344 sha256=c4fd791f5282a634204a9b549db10c657d958e49bd843e6f9b0e615200e673c1
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext
     |████████████████████████████████| 316 kB 5.5 MB/s 
     |████████████████████████████████| 1.4 MB 44.2 MB/s 
     |████████████████████████████████| 233 kB 53.2 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=e2f4cd110acf5dae45561cf85c35aee4453ad33f44b04422e19880010f0b7016
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154351 sha256=82568215a00da239ffffc9edbb24c343ac02939910a7605e5a014

[download data](https://www.digikala.com/opendata/#section-4)

In [4]:
import pandas as pd
# download data => https://www.digikala.com/opendata/#section-4
df = pd.read_excel('/content/drive/MyDrive/data/digikala_data/digikala.xlsx')
df.comment = df.comment.apply(str)
df.head()

,product_id,product_title,title_en,user_id,likes,dislikes,verification_status,recommend,title,comment,advantages,disadvantages
0,3692,ماوس بی‌سیم لاجیتک مدل M325,IT,989472,0,0,verified,\N,NaN,واقعا عالیه. من که ازش خیلی راضیم,NaN,NaN
1,90213,شارژر همراه شیاومی مدل NDY-02-AN با ظرفیت 1000...,AC,3862150,4,1,verified,recommended,واقعاً عالیه,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,"[""عمر طولانی\r"",""افت بسیار کم میزان شارژ\r"",""ا...","[""ندارد""]"
2,59473,یدک پولیشر میکروفایبر مهسان مدل 20119,HW,626843,1,0,verified,not_recommended,خیلی سخت حوله اش در میاد,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...,NaN,NaN
3,120499,گوشی موبایل هوآوی آنر مدل 5X KIW-L21 دو سیم‌کارت,MO,786887,6,11,verified,no_idea,گوشی مقرون به صرفه,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...,"[""صفحه نمایش پرنور و با کیفیت\r"",""کیفیت بالای ...","[""کیفیت پایین اسپیکر\r"",""حاشیه خالی زیر صفحه ن..."
4,67200,شارژر همراه شیائومی مدل Mi ظرفیت 5000 میلی آمپ...,AC,854531,19,4,verified,no_idea,ابعاد، استحکام و نگهداری شارژ مناسب,اگر ظرفیتش براتون کافیه حتما بخرید.\r\nیه شارژ...,"[""ظریف و زیبا و باریک\r"",""بدنه محکم و با دوام""]","[""ظرفیت متوسط""]"


In [5]:
df.groupby('recommend').count()

,product_id,product_title,title_en,user_id,likes,dislikes,verification_status,title,comment,advantages,disadvantages
recommend,,,,,,,,,,,
\N,36382,36382,36382,36382,36382,36382,36382,35106,36382,16241,11953
no_idea,10536,10536,10536,10536,10536,10536,10536,10207,10536,3840,3746
not_recommended,16110,16110,16110,16110,16110,16110,16110,15363,16110,4881,6256
recommended,36972,36972,36972,36972,36972,36972,36972,36054,36972,17279,12166


In [6]:
df.recommend[df.recommend == '\\N'] = 2
df.recommend[df.recommend == 'no_idea'] = 2
df.recommend[df.recommend == 'not_recommended'] = 3
df.recommend[df.recommend == 'recommended'] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [7]:
df_final = df[['recommend', 'comment']]
df_final.head()

,recommend,comment
0,2,واقعا عالیه. من که ازش خیلی راضیم
1,1,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...
2,3,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...
3,2,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...
4,2,اگر ظرفیتش براتون کافیه حتما بخرید.\r\nیه شارژ...


### preprocessing data 

In [8]:
import fasttext 

%time
model = fasttext.load_model("/content/drive/MyDrive/data/cc.fa.300.bin")

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.82 µs


In [9]:
def CleanPersianText(text):
  _normalizer = hazm.Normalizer()
  text = _normalizer.normalize(text)
  return text

revlist = list(map(lambda x: [CleanPersianText(x[0]),x[1]],zip(df['comment'],df['recommend'])))
pos=list(filter(lambda x: x[1] == 1,revlist))
nat=list(filter(lambda x: x[1] == 2,revlist))
neg=list(filter(lambda x: x[1] == 3,revlist))

revlist_shuffle = pos[:450] + neg[:450]
random.shuffle(revlist_shuffle)

print("Posetive count {}".format(len(pos)))
print("Negetive count {}".format(len(neg)))
print("Natural  count {}".format(len(nat)))
print()
print("Total    count {}".format(len(revlist)))
print()
print("Posetive count : ","\n",pos[random.randrange(1,len(pos))])
print("Negetive count : ","\n",neg[random.randrange(1,len(neg))])
print("unknown  count : ","\n",nat[random.randrange(1,len(nat))])
print("Total    count {}".format(len(revlist_shuffle)))

Posetive count 36972
Negetive count 16110
Natural  count 46918

Total    count 100000

Posetive count :  
 ['خیلی راحت و بدون فشار دیوار رو سوراخ می\u200cکنه', 1]
Negetive count :  
 ['چیزی که در تعریف این محصول درباره محفظه نگهدارینده نوشته: «محفظه\u200cی اشکی\u200cمانند ماسک پیوردرم از جنس شیشه\u200cی شفاف ساخته شده و در آن به\u200cصورت چوب\u200cپنبه\u200cای است.» اصلا صحت نداره و بسته بندی کاغذیه و به هیچ وجه به شکلی که تو توضیحات اشاره شده شباهت نداره.', 3]
unknown  count :  
 ['کیفیت ماگ پایین بود هم بامبو هم خود ماگ ولی شیکه', 2]
Total    count 900


### train test split

In [10]:
vector_size = 300 
max_no_tokens = 20 
import numpy as np
import keras.backend as K
train_size = int(0.8*(len(revlist_shuffle)))
test_size = int(0.2*(len(revlist_shuffle)))#lstm
indexes = set(np.random.choice(len(revlist_shuffle), train_size + test_size, replace=False))

In [11]:
x_train = np.zeros((train_size, max_no_tokens, vector_size), dtype=K.floatx())
y_train = np.zeros((train_size, 2), dtype=np.int32)

x_test = np.zeros((test_size, max_no_tokens, vector_size), dtype=K.floatx())
y_test = np.zeros((test_size, 2), dtype=np.int32)

In [12]:
index_test = []
for i, index in enumerate(indexes):
  text_words = hazm.word_tokenize(revlist_shuffle[index][0])
  for t in range(0,len(text_words)):
    if t >= max_no_tokens:
      break

    if text_words[t] not in model.words:
      continue
    if i < train_size:
      x_train[i, t, :] = model.get_word_vector(text_words[t])
    else:
      x_test[i - train_size, t, :] = model.get_word_vector(text_words[t])

  if i < train_size:
    y_train[i, :] = [1.0, 0.0] if revlist_shuffle[index][1] == 3 else [0.0, 1.0]
  else:
    y_test[i - train_size, :] = [1.0, 0.0] if revlist_shuffle[index][1] == 3 else [0.0, 1.0]
    index_test.append(index)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((720, 20, 300), (180, 20, 300), (720, 2), (180, 2))

### select index of rule 

In [13]:
text_rule = []

for x in index_test:
  text_rule.append(revlist_shuffle[x][0])

### Run rules

In [14]:
lemmatizer = Lemmatizer()
tagger = POSTagger(model='resources/postagger.model')
lexicon = pd.read_excel("/content/drive/MyDrive/PerSent.xlsx")

li_key = []
li_value = []
keyword_list = ['اما' , 'اگرچه' , 'با اینکه', 'ولی']
keyword_list2 = ['حالیکه', 'صورتیکه']

for i in range(len(text_rule)):
  text = text_rule[i]
  tag = tagger.tag(word_tokenize(text))
  # rule 2  -----------------------------------------------------------------------------  rule 2 
  value_polarity = []
  if 'که' in text:
    #print(text.split('که')[0])
    target = text.split('که')[0]
    tag_target = tagger.tag(word_tokenize(target))
    #print(tag_target)
    lem_target_tag = [] 
    for x in tag_target:
      lem_target_tag.append(lemmatizer.lemmatize(x[0]))
    #print('lem_target_tag',lem_target_tag)
    for lem in lem_target_tag:
      #به ازای هر کلمه ای که در لکسیکون داریم
      df_have = lexicon.loc[lexicon['Words'] == lem]
      #print('df_have', df_have)
      #print('len df_have', len(df_have))
      if len(df_have) > 1:
        #print('>1')
        value_have = df_have["Polarity"].mean()
        
        value_polarity.append(value_have)
      elif len(df_have) == 1:
        #print('=1')
        value_have = df_have['Polarity']
        if value_have.values[0] != 0:
          value_polarity.append(value_have.values[0])
    #print(value_polarity)
    if len(value_polarity) > 0:
      
      li_key.append(i)
      li_value.append(sum(value_polarity) / len(value_polarity))
# rule 2  -----------------------------------------------------------------------------  rule 2 


# rule 7  -----------------------------------------------------------------------------  rule 7 

  if 'مخالف' in text: 
    
    li_key.append(i)
    li_value.append(-1)
# rule 7  -----------------------------------------------------------------------------  rule 7 
 
# rule 3  -----------------------------------------------------------------------------  rule 3 
  value_polarity = []
  if [key for key in keyword_list if key in text ]:
    new_keyword_list = [key for key in keyword_list if key in text ]

    value_polarity = []
    #print(new_keyword_list,CONDITION)
    for x in new_keyword_list:
      value_polarity = []
      #print(text.split('که')[0])
      target = text.split(x)[1]
      tag_target = tagger.tag(word_tokenize(target))
      #print(tag_target)
      lem_target_tag = [] 
      for x in tag_target:
        lem_target_tag.append(lemmatizer.lemmatize(x[0]))
      #print('lem_target_tag',lem_target_tag)
      for lem in lem_target_tag:
        #به ازای هر کلمه ای که در لکسیکون داریم
        df_have = lexicon.loc[lexicon['Words'] == lem]
        #print('df_have', df_have)
        #print('len df_have', len(df_have))
        if len(df_have) > 1:
          #print('>1')
          value_have = df_have["Polarity"].mean()
          
          value_polarity.append(value_have)
        elif len(df_have) == 1:
          #print('=1')
          value_have = df_have['Polarity']
          if value_have.values[0] != 0:
            value_polarity.append(value_have.values[0])

    if len(value_polarity) > 0:
      
      li_key.append(i)
      li_value.append(sum(value_polarity) / len(value_polarity))
# rule 3  -----------------------------------------------------------------------------  rule 3

# rule 4  -----------------------------------------------------------------------------  rule 4 
  value_polarity = []
  if [key for key in keyword_list2 if key in text ]:
    new_keyword_list2 = [key for key in keyword_list2 if key in text ]

    value_polarity = []
    #print(new_keyword_list2,CONDITION)
    for x in new_keyword_list2:
      value_polarity = []
      #print(text.split('که')[0])
      target = text.split(x)[1]
      tag_target = tagger.tag(word_tokenize(target))
      #print(tag_target)
      lem_target_tag = [] 
      for x in tag_target:
        lem_target_tag.append(lemmatizer.lemmatize(x[0]))
      #print('lem_target_tag',lem_target_tag)
      for lem in lem_target_tag:
        #به ازای هر کلمه ای که در لکسیکون داریم
        df_have = lexicon.loc[lexicon['Words'] == lem]
        #print('df_have', df_have)
        #print('len df_have', len(df_have))
        if len(df_have) > 1:
          #print('>1')
          value_have = df_have["Polarity"].mean()
          
          value_polarity.append(value_have)
        elif len(df_have) == 1:
          #print('=1')
          value_have = df_have['Polarity']
          if value_have.values[0] != 0:
            value_polarity.append(value_have.values[0])

    if len(value_polarity) > 0:
      
      li_key.append(i)
      li_value.append(sum(value_polarity) / len(value_polarity))
# rule 4  -----------------------------------------------------------------------------  rule 4

# rule 5  -----------------------------------------------------------------------------  rule 5 
  if [x for x in tag if 'که' in x ]:
    index_list = [tag.index(x) for x in tag if 'که' in x ]
    value_polarity = []
    idx = index_list[0]
    if len(tag)> idx+1 and'این' in tag[idx+1]:
      #print(text.split('این')[1])
      target = text.split('این')[1]
      value_polarity = []
      #print(text.split('که')[0])
      tag_target = tagger.tag(word_tokenize(target))
      #print(tag_target)
      lem_target_tag = [] 
      for x in tag_target:
        lem_target_tag.append(lemmatizer.lemmatize(x[0]))
      #print('lem_target_tag',lem_target_tag)
      for lem in lem_target_tag:
        #به ازای هر کلمه ای که در لکسیکون داریم
        df_have = lexicon.loc[lexicon['Words'] == lem]
        #print('df_have', df_have)
        #print('len df_have', len(df_have))
        if len(df_have) > 1:
          #print('>1')
          value_have = df_have["Polarity"].mean()
          
          value_polarity.append(value_have)
        elif len(df_have) == 1:
          #print('=1')
          value_have = df_have['Polarity']
          if value_have.values[0] != 0:
            value_polarity.append(value_have.values[0])
      
    if len(value_polarity) > 0:
      
      li_key.append(i)
      li_value.append(sum(value_polarity) / len(value_polarity))
  
  # rule 5  -----------------------------------------------------------------------------  rule 5

  # rule 9  -----------------------------------------------------------------------------  rule 9
  if [x for x in tag if x[1] == 'AJ' ]:
    word = [x[0] for x in tag if x[1] == 'AJ' ]
    #print(word)
    negative_word = [x for x in word if x.startswith("نا") or x.startswith("بی") or x.startswith("ضد") or x.startswith("زهر") or x.startswith("لا")]
    #print(negative_word)
    z = [x for x in negative_word if 'بیش' not in x]

    for x in z:
      if len(x) > 1:
        
        li_key.append(i)
        li_value.append(-1)
    #print(i)
    #if word[0].startswith("نا") or word[0].startswith("بی") or word[0].startswith("ضد") or word[0].startswith("زهر") or word[0].startswith("لا"):
      #if word[0] != 'بیش' and word[0] != 'بیشتر' and word[0] != 'لازم' and word[0] != 'بیشتری' and word[0] != 'لایک' :
        #print(x,lemmatizer.lemmatize(x[0]))
        #d[i] = -1
        #li_key.append(i)
        #li_value.append(-1)
  # rule 9  -----------------------------------------------------------------------------  rule 9
  
  
        

In [15]:
#li_key 5 = yani too index_rule khooneh 5 esh ghabool shode
import numpy as np

df = pd.DataFrame({'li_key':li_key , 'li_value':li_value})
df_gg = df.loc[df.groupby('li_key')['li_value'].apply(lambda x: x.abs().idxmax())]
df_gg.loc[df_gg.li_value < 0, 'li_value'] = 0
df_gg.loc[df_gg.li_value > 0, 'li_value'] = 1
key = []
for x in df_gg['li_key'].tolist():
  key.append(index_test[x])
  
df_key = pd.DataFrame(
    {
      'li_key': key,
     'li_value': df_gg['li_value'].tolist()
    })

index_shuffle = []

for x in df_key['li_key'].tolist():
  index_shuffle.append(revlist_shuffle[x][1])

result = df_key.join(pd.DataFrame(index_shuffle))

result.rename(columns={0:'true_value'}, inplace=True)

result.loc[result.true_value == 3, 'true_value'] = 0
result.loc[result.true_value == 1, 'true_value'] = 1

from sklearn.metrics import accuracy_score
y_pred = result['li_value'].tolist()
y_true = result['true_value'].tolist()
print(accuracy_score(y_true, y_pred))
print(accuracy_score(y_true, y_pred, normalize=False))
none_polarity = list(set(index_test) - set(key))
print(len(none_polarity))

0.6923076923076923
36
128


### select index of CNN

In [16]:
x_none_polarity = np.zeros((len(none_polarity), max_no_tokens, vector_size), dtype=K.floatx())
y_none_polarity = np.zeros((len(none_polarity), 2), dtype=np.int32)

In [17]:
vector_size = 300 
max_no_tokens = 20 
for i, index in enumerate(set(none_polarity)):
  text_words = hazm.word_tokenize(revlist_shuffle[index][0])
 
  for t in range(0,len(text_words)):
    if t >= max_no_tokens:
      break
    
    if text_words[t] not in model.words:
      continue
      
    x_none_polarity[i , t, :] = model.get_word_vector(text_words[t])
    

  
  y_none_polarity[i , :] = [1.0, 0.0] if revlist_shuffle[index][1] == 3 else [0.0, 1.0]
    
x_none_polarity.shape,y_none_polarity.shape

((128, 20, 300), (128, 2))

In [18]:
import numpy as np
xx_test = np.concatenate([x_none_polarity,x_test])
yy_test = np.concatenate([y_none_polarity,y_test])

### CNN model

In [19]:
batch_size = 500 
w2v_model = model
no_epochs = 200
del model

In [20]:
from keras.models import Sequential
from keras.layers import Conv1D, Dropout, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, TensorBoard


model = Sequential()

model.add(Conv1D(128, kernel_size=5, activation='relu', padding='same',input_shape=(max_no_tokens, vector_size)))
model.add(Dropout(0.5))
model.add(Conv1D(64, kernel_size=5, activation='relu', padding='same'))
model.add(Dropout(0.5))
model.add(Conv1D(32, kernel_size=5, activation='relu', padding='same'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

# tensorboard = TensorBoard(log_dir='logs/', histogram_freq=0, write_graph=True, write_images=True)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 20, 128)           192128    
                                                                 
 dropout (Dropout)           (None, 20, 128)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 20, 64)            41024     
                                                                 
 dropout_1 (Dropout)         (None, 20, 64)            0         
                                                                 
 conv1d_2 (Conv1D)           (None, 20, 32)            10272     
                                                                 
 dropout_2 (Dropout)         (None, 20, 32)            0         
                                                                 
 flatten (Flatten)           (None, 640)               0

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


### Run CNN & rule

In [21]:
model.fit(x_train, y_train, batch_size=batch_size, shuffle=True, epochs=no_epochs,
         validation_data=(xx_test, yy_test))

Epoch 1/200
2/2 [==============================] - 2s 489ms/step - loss: 0.6947 - accuracy: 0.5097 - val_loss: 0.6924 - val_accuracy: 0.4903
Epoch 2/200
2/2 [==============================] - 1s 225ms/step - loss: 0.6990 - accuracy: 0.4764 - val_loss: 0.6924 - val_accuracy: 0.4935
Epoch 3/200
2/2 [==============================] - 1s 230ms/step - loss: 0.6974 - accuracy: 0.4903 - val_loss: 0.6923 - val_accuracy: 0.5065
Epoch 4/200
2/2 [==============================] - 1s 223ms/step - loss: 0.6974 - accuracy: 0.4792 - val_loss: 0.6922 - val_accuracy: 0.5000
Epoch 5/200
2/2 [==============================] - 1s 216ms/step - loss: 0.6936 - accuracy: 0.5222 - val_loss: 0.6922 - val_accuracy: 0.4935
Epoch 6/200
2/2 [==============================] - 1s 221ms/step - loss: 0.6861 - accuracy: 0.5236 - val_loss: 0.6921 - val_accuracy: 0.4968
Epoch 7/200
2/2 [==============================] - 1s 222ms/step - loss: 0.6971 - accuracy: 0.4944 - val_loss: 0.6920 - val_accuracy: 0.4968
Epoch 8/200
2

In [22]:
model.evaluate(x=xx_test, y=yy_test, batch_size=32, verbose=1)

10/10 [==============================] - 0s 9ms/step - loss: 0.5396 - accuracy: 0.7825


[0.5396364331245422, 0.7824675440788269]

In [23]:
pred = model.predict(xx_test[:])
yy_pred = []
for x in pred:
  if x[0] > x[1]:
    yy_pred.append(0)
  else :
    yy_pred.append(1)

pred_true = []
for x in yy_test:
  if x[0] == 1:
    pred_true.append(0)
  else:
    pred_true.append(1)
y_pred = result['li_value'].tolist() + yy_pred
y_true = result['true_value'].tolist() + pred_true
from sklearn.metrics import accuracy_score
accuracy_score(list(y_true), list(y_pred))

0.7694444444444445

### Run just CNN

In [24]:
model.fit(x_train, y_train, batch_size=batch_size, shuffle=True, epochs=no_epochs,
         validation_data=(x_test, y_test))

Epoch 1/200
2/2 [==============================] - 1s 253ms/step - loss: 0.2456 - accuracy: 0.9097 - val_loss: 0.5713 - val_accuracy: 0.7722
Epoch 2/200
2/2 [==============================] - 1s 232ms/step - loss: 0.2483 - accuracy: 0.9056 - val_loss: 0.5668 - val_accuracy: 0.7611
Epoch 3/200
2/2 [==============================] - 1s 201ms/step - loss: 0.2240 - accuracy: 0.9153 - val_loss: 0.5670 - val_accuracy: 0.7667
Epoch 4/200
2/2 [==============================] - 1s 210ms/step - loss: 0.2353 - accuracy: 0.9167 - val_loss: 0.5684 - val_accuracy: 0.7611
Epoch 5/200
2/2 [==============================] - 1s 210ms/step - loss: 0.2301 - accuracy: 0.9139 - val_loss: 0.5736 - val_accuracy: 0.7667
Epoch 6/200
2/2 [==============================] - 0s 205ms/step - loss: 0.2176 - accuracy: 0.9181 - val_loss: 0.5730 - val_accuracy: 0.7667
Epoch 7/200
2/2 [==============================] - 1s 197ms/step - loss: 0.2168 - accuracy: 0.9222 - val_loss: 0.5714 - val_accuracy: 0.7667
Epoch 8/200
2

In [25]:
model.evaluate(x=x_test, y=y_test, batch_size=32, verbose=1)

6/6 [==============================] - 0s 9ms/step - loss: 0.9315 - accuracy: 0.7667


[0.9314975142478943, 0.7666666507720947]

In [26]:
pred = model.predict(x_test[:])
yy_pred = []
for x in pred:
  if x[0] > x[1]:
    yy_pred.append(0)
  else :
    yy_pred.append(1)

pred_true = []
for x in y_test:
  if x[0] == 1:
    pred_true.append(0)
  else:
    pred_true.append(1)

from sklearn.metrics import accuracy_score
accuracy_score(list(pred_true), list(yy_pred))

0.7666666666666667